In [2]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
for row in mi.rows:
    print(row.original_coordinates)

[5651, 230, 3450]
[5901, 650, 6670]
[7426, 245, 3840]
[4000, 200, 7800]
[4333, 224, 7068]
[0, 2000, 1600]
[4761, 4952, 6493]
[7000, 4151, 9017]
[4275, 7350, 5700]
[4443, 5716, 3322]


In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
from annotator_metrics.src.preprocessing import copy_data
mi = MaskInformation()
print(mi.rows)
copy_data(group = "group1", output_base_path="/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/")

In [2]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
print(mi.rows[0].organelle_info)


{'mito_mem ': 3, 'mito_lumen ': 4, 'mito': 0}


In [3]:
import numpy as np
a=np.random.random((4,4,4))

In [7]:
print(a[1:3,1:3,1:3])

[[[0.29667459 0.14585225]
  [0.93038333 0.71463271]]

 [[0.04604659 0.92718152]
  [0.80627626 0.55996372]]]


In [2]:
from annotator_metrics.src.metrics_evaluations import calculate_all_to_all
results= calculate_all_to_all("group1","/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/", num_workers = 10)
print(results[1:10])

distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-bsi116em', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-9nhp__t8', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-h97b3p_6', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-wjwehcyw', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-bcgl7qtp', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/

http://127.0.0.1:8787/status
([['01', 'mito_mem ', 'F1 Score', 0, 1, 0.8911146164767753], ['01', 'mito_mem ', 'Jaccard Index', 0, 1, 0.8036129159223528], ['01', 'mito_mem ', 'Hausdorff Distance', 0, 1, 43.86342439892262], ['01', 'mito_mem ', 'False Negative Rate', 0, 1, 0.12671622807046493], ['01', 'mito_mem ', 'False Negative Rate with Tolerance Distance', 0, 1, 0.0], ['01', 'mito_mem ', 'False Positive Rate', 0, 1, 0.04895570140406419], ['01', 'mito_mem ', 'False Discovery Rate', 0, 1, 0.0903112157443002], ['01', 'mito_mem ', 'False Positive Rate with Tolerance Distance', 0, 1, 1.6772288601142677e-05], ['01', 'mito_mem ', 'Variation Of Information', 0, 1, 0.7687484070183797], ['01', 'mito_mem ', 'Mean False Distance', 0, 1, 0.5416146748376212], ['01', 'mito_mem ', 'Mean False Negative Distance', 0, 1, 0.540721377626499], ['01', 'mito_mem ', 'Mean False Positive Distance', 0, 1, 0.5425079720487435], ['01', 'mito_mem ', None, 0, 1, 0.5416146748376212], ['01', 'mito_mem ', 'Mean False N

In [3]:
type(results)

tuple

In [11]:
import time
import tifffile
from dask_image import imread
start = time.time(); temp = imread.imread('/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/group1/01/a1.tif'); print("loading: ", time.time() - start)
print(temp.shape)


loading:  0.008377313613891602
(122, 186, 185)


In [6]:
import dask
import dask.distributed
dask.config.set({"distributed.comm.timeouts.connect":100})
dask.config.get("distributed.comm.timeouts.connect")

100